## Module submission header

### Submission preparation instructions 
Completion of this header is mandatory, subject to a 2-point deduction to the assignment._ Only add plain text in the designated areas, i.e., replacing the relevant 'NA's. You must fill out all group member Names and Drexel email addresses in the below markdown list, under header __Module submission group__. It is required to fill out descriptive notes pertaining to any tutoring support received in the completion of this submission under the __Additional submission comments__ section at the bottom of the header. If no tutoring support was received, leave NA in place. You may as well list other optional comments pertaining to the submission at bottom. _Any disruption of this header's formatting will make your group liable to the 2-point deduction.

### Module submission group
- Group member 1     
    - Name:Yiyun Fan   
    - Email: yf366@drexel.edu  
    
- Group member 2    
    - Name: Shreekant Malviya    
    - Email: sm4546@drexel.edu
    
- Group member 3    
    - Name: Kunal Sharma   
    - Email: kos26@drexel.edu
- Group member 4    
    - Name: NA     
    - Email: NA  
    
### Additional submission comments 
- Tutoring support received: NA
- Other (other): NA 

# Assignment group 1: Textual feature extraction and numerical comparison

## Module C _(35 points)_ Similarity of word usage across a document

Here we'll be building up some code to discover how different terms are utilized similarly across a document. For this, our first task will be to create a word frequency counting function.

__C1.__ _(12 points)_ Define a function called `count_words(paragraph, pos = True, lemma = True)` that `return`s a `Counter()` called `frequency`. In `frequency`, each key will consist of a `heading = (text, tag)`, where `text` contains the `word.text` attribute from `spacy` if `lemma = False`, and `word.lemma_` attribute if `True`. Similarly, `tag` should be left empty as `""` if `pos = False` and otherwise contain `word.pos_`. The `Counter()` should simply contain the number of times each `heading` is observed in the `paragraph`.

In [1]:
# C1:Function(12/12)

from collections import Counter
import spacy, json, re

nlp = spacy.load('en')

def count_words(paragraph, pos = True, lemma = True):

    doc = nlp(paragraph)
    
    frequency = Counter()
    for sentence in doc.sents:
        for word in sentence:
            heading = (word.lemma_ if lemma else word.text, 
                       word.pos_ if pos else "")
        
            frequency[heading] += 1
    return frequency

Let's make sure your function works by testing it on a short sentence. 

In [2]:
# C1:SanityCheck

count_words("The quick brown fox jumps over the lazy dog.")

Counter({('the', 'DET'): 2,
         ('quick', 'ADJ'): 1,
         ('brown', 'ADJ'): 1,
         ('fox', 'PROPN'): 1,
         ('jump', 'VERB'): 1,
         ('over', 'ADP'): 1,
         ('lazy', 'ADJ'): 1,
         ('dog', 'NOUN'): 1,
         ('.', 'PUNCT'): 1})

__C2.__ _(8 pts)_ Next, define a function called `book_TDM(book_id, pos = True, lemma = True)` and copy into it the TDM-producing code from __Section 2.1.5.1__ of the lecture notes, now `return`-ing `TDM` and `all_words`. Once copied, modify this function to call `count_words` appropriately, now passing through the user of `book_TDM`'s specified `lemma` and `pos` arguments.

In [3]:
# C2:Function(8/8)

import numpy as np
from collections import Counter
import re

def book_TDM(book_id, pos = True, lemma = True):

    ## loading the book
    with open("./data/books/"+book_id+".txt") as file:
        book = file.read().strip()
    
    ## breaking the document into paragraphs
    paragraphs = re.split("\n\n+", book)

    ## keeping track of the words in all documents by the 'master' set
    all_words = set()

    ## Storing the word frequencies by book
    all_doc_frequencies = {}

   
    for j, paragraph in enumerate(paragraphs):        
        frequency = count_words(paragraph, pos = pos, lemma = lemma)
        all_doc_frequencies[j] = frequency
        doc_words = set(frequency.keys())
        all_words = all_words.union(doc_words)

    ## creating a matrix of zeros: (words) x (documents)
    TDM = np.zeros((len(all_words),len(all_doc_frequencies)))
    ## fixing a word order for the rows
    all_words = sorted(list(all_words))
    ## looping over the (sorted) document numbers and (ordered) words; fill in matrix
    for j in all_doc_frequencies:
        for i, word in enumerate(all_words):
            TDM[i,j] = all_doc_frequencies[j][word]

    return(TDM, all_words)


To test your code's function, let's process `book_id = 84` with both of `pos = True` and `lemma = True` and print out the `TDM`'s `.shape` attribute and the first ten terms in `all_words`.

In [4]:
# C2:SanityCheck

TDM, terms = book_TDM("84", pos = True, lemma = True)
terms[:10]

[('\n', 'SPACE'),
 ('\n  ', 'SPACE'),
 ('\n   ', 'SPACE'),
 ('\n     ', 'SPACE'),
 ('\n                              ', 'SPACE'),
 (' ', 'SPACE'),
 ('  ', 'SPACE'),
 ('    ', 'SPACE'),
 ('     ', 'SPACE'),
 ('               ', 'SPACE')]

In [5]:
# C2:SanityCheck

TDM.shape

(6256, 723)

__C3.__ _(8 pts)_ Next, your job is to define two functions. The first is `sim(u,v)`, which shoud take two arbitrary numeric vectors and compute/output the `cosine_similarity`, as described in __Section 1.1.2.10__.  

The second function is `term_sims(i, TDM)`, which should utilize the first function (`sim` function) to output a list of cosine similarity values (`sim_values`) between the word/row `i` and all others (rows) in the `TDM`.

Note: each of these functions can be straightforwardly completed using a single line of code! Exhibit your knowledge of comprehensions and vectorization!

In [6]:
# C3:Function(4/8)
def sim(u,v):
    
    cosine_similarity= u.dot(v) / (np.linalg.norm(u) * np.linalg.norm(v))
    
    return cosine_similarity

In [7]:
# C3:SanityCheck

print("Exactly similar:", sim(np.array([1,2,3]), np.array([1,2,3])))
print("Exactly dissimilar:", sim(np.array([1,2,3]), np.array([-1,-2,-3])))
print("In the middle:", sim(np.array([1,1]), np.array([-1,1])))

Exactly similar: 1.0
Exactly dissimilar: -1.0
In the middle: 0.0


In [8]:
# C3:Function(4/8)

def term_sims(i, TDM):
    
    sim_values=  [sim(TDM[i,], TDM[sim_i,]) for sim_i in range(TDM.shape[0])]
    
    return sim_values

In [9]:
# C3:SanityCheck

# Compare word/row 0 to all other (rows) in the TDM
term_sims(0, TDM)

[1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.9498986502907771,
 0.045949296887323965,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.3952794253601441,
 0.46742415729096676,
 0.24336126093426197,
 0.10849610772519391,
 0.4329628779238955,
 0.2063477173114049,
 0.2063477173114049,
 0.9523276661400791,
 0.061647450815025526,
 0.14726891028033876,
 0.4905429499901957,
 0.027398867028900233,
 0.41119037303226363,
 0.023974008650287704,
 0.0,
 0.058222592436413,
 0.9248705442120388,
 0.9450169601805637,
 0.9712507693137749,
 0.0,
 0.0,
 0.0,
 0.029060887009445802,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.010274575135837589,
 0.0,
 0.0,
 0.0,
 0.013699433514450117,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.015818743254626313,
 0.0,
 0.0,
 0.041098300543350355,
 0.35580586003870596,
 0.8189613459883718,
 0.3762721959394875,
 0.0,
 0.06920700173899012,
 0.016952184088843386,
 0.07534688432947564,
 0.14118761213114184,
 0.16510813272016214,
 0.00342485837861252

__C4.__ _(7 pts)_ Finally, your goal now is to a write function, `most_similar(term, terms, TDM, top = 25)`, that utilizes `term_sims` to output a sorted list of the `top = N` terms (`top_n_terms`) most similar to one specified (`term`). The output data type should be a list of lists, with each inner list representing information for a similar term as: `[row_ix, similarity, term]`. 

\[Hint: to locate the row containing the term of interest, utilize the list `.index()` method in application to the `terms` argument.\]

In [10]:
# C4:Function(6/7)

def most_similar(term, terms, TDM, top = 25):
    
    top_n_terms=[]
    term_index = terms.index(term)
    sims = term_sims(term_index, TDM)
    sims = sorted(enumerate(sims), key = lambda x: x[1], reverse = True)
    for sim_i, term_sim in sims[:25]:
        top_n_terms.append((sim_i, term_sim, terms[sim_i]))
    
    return top_n_terms

Now, let's test your functions utility on a `TDM` produced for `book_id = 84` and exhibit the top 25 similar terms to both of `('monster', 'NOUN')` and `('beautiful', 'ADJ')`.

In [11]:
# C4:SanityCheck

most_similar(('monster', 'NOUN'), terms, TDM, top = 25)

[(3782, 1.0, ('monster', 'NOUN')),
 (413, 0.3380617018914066, ('Would', 'VERB')),
 (778, 0.3380617018914066, ('asseveration', 'NOUN')),
 (1526, 0.3380617018914066, ('correct', 'VERB')),
 (2283, 0.3380617018914066, ('existence', 'VERB')),
 (2566, 0.3380617018914066, ('formation', 'NOUN')),
 (3851, 0.3380617018914066, ('mutilate', 'VERB')),
 (4360, 0.3380617018914066, ('posterity', 'NOUN')),
 (29, 0.30686479781192966, ('-PRON-', 'PRON')),
 (2918, 0.30578831486257535, ('hideous', 'ADJ')),
 (5, 0.2929277805918349, (' ', 'SPACE')),
 (1734, 0.29277002188455997, ('demoniacal', 'ADJ')),
 (2560, 0.29277002188455997, ('forgetfulness', 'NOUN')),
 (12, 0.28811865589289226, ('!', 'PUNCT')),
 (5550, 0.2825393662324921, ('that', 'SCONJ')),
 (30, 0.27266231928075757, ('.', 'PUNCT')),
 (0, 0.27092834781732605, ('\n', 'SPACE')),
 (660, 0.2680355919935828, ('and', 'CCONJ')),
 (5552, 0.2665315201084469, ('the', 'DET')),
 (3892, 0.2548235957188128, ('neck', 'NOUN')),
 (3865, 0.253546276418555, ('narrative'

In [12]:
# C4:SanityCheck

most_similar(('beautiful', 'ADJ'), terms, TDM, top = 25)

[(906, 0.9999999999999998, ('beautiful', 'ADJ')),
 (331, 0.37499999999999994, ('Rotterdam', 'PROPN')),
 (1163, 0.37499999999999994, ('castle', 'NOUN')),
 (5107, 0.37499999999999994, ('singularly', 'ADV')),
 (6102, 0.37499999999999994, ('whence', 'NOUN')),
 (249, 0.35355339059327373, ('Mainz', 'PROPN')),
 (250, 0.35355339059327373, ('Mannheim', 'PROPN')),
 (941, 0.35355339059327373, ('beneath', 'ADV')),
 (1724, 0.35355339059327373, ('delineate', 'PROPN')),
 (2024, 0.35355339059327373, ('dun', 'PROPN')),
 (2457, 0.35355339059327373, ('fifth', 'NOUN')),
 (3594, 0.35355339059327373, ('luxuriance', 'NOUN')),
 (3671, 0.35355339059327373, ('meander', 'VERB')),
 (4188, 0.35355339059327373, ('pearly', 'ADV')),
 (4341, 0.35355339059327373, ('populous', 'ADJ')),
 (5043, 0.35355339059327373, ('ship', 'VERB')),
 (5065, 0.35355339059327373, ('shrivel', 'VERB')),
 (5141, 0.35355339059327373, ('sloping', 'NOUN')),
 (5305, 0.35355339059327373, ('steep', 'ADJ')),
 (5329, 0.35355339059327373, ('straight'

In [13]:
# C4:Inline

# Comment on the ordered results returned in the sanity checks.
# Do you think the algorithm is exhibiting sensible results? print "Yes" or "No"
print("Yes")

Yes
